In [ ]:
#CLEAN and CREATE DATA
#1. Packages and paths
#2. Erstellen small Matrix from Excel (Aktivitäten (Fahre, Stehen, Lenkpause), Uhrzeiten)
#3. Addressen Matrix von Excel erstellen
#3.1. Leere Adressen Matrix erstellen und Funktionen
#3.2. Addressen Matrix befüllen (jedes Fahrzeug einzeln hinzufügen, um zu debuggen & wegen langer Zeit)
#3.3. Fehler korriegieren und Speichern
#4. big Matrix von Excel erstellen (Aktivitäten, Uhrzeiten, Standort, Distanz, Beladung, Höhenmeter)
#4.1. big Matrix ohne Adressen erstellen
#4.2.Adressen, Distanzen, Höhenunterschied und Geschwindigkeit hinzufügen
#5. Kennzahlen (Durchschnite, Maxima und Minima)

In [ ]:
#1. PACKAGES AND PATHS

#!pip install pandas matplotlib
#!pip install openrouteservice
import pandas as pd
import numpy as np
import six
import matplotlib.pyplot as plt
from datetime import datetime
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
import openrouteservice
from openrouteservice import convert
import requests
import json
import re
import os
import pickle
import math
import matplotlib.cm as cm
api_key = 'api_key' #muss ich immer wieder neu erstellt werden über account: https://openrouteservice.org/dev/#/home
client = openrouteservice.Client(key=api_key)

FZG =  ['WT-BR 270','WT-BR 231','WT-BR240', 'WT-BR 391E ', 'FR-BR 1315', 'WT-BR 229', 'WT-BR 228', 'WT-BR 2551', 'WT-BR 608', 'WT-BR 213', 'WT-BR 225', 'WT-BR 253', 'FR-BR 1317']
new_label={'WT-BR 231': 'fzg a','WT-BR240': 'fzg b', 'WT-BR 391E ': 'fzg c', 'FR-BR 1315': 'fzg d', 'WT-BR 270': 'fzg e', 'WT-BR 229': 'fzg f', 'WT-BR 228': 'fzg g', 'WT-BR 2551': 'fzg h', 'WT-BR 608': 'fzg i', 'WT-BR 213': 'fzg j', 'WT-BR 225': 'fzg k', 'WT-BR 253': 'fzg l', 'FR-BR 1317': 'fzg m'}

# datei öffnen
path = 'Zugang zur Excel Tabelle' # Daten
save_path= 'Speicherort' #Speicherortrt

In [ ]:
# 2. Small Matrix (für jedes Fahrzeug und jeden Tag wird eine Liste erstellt mit allen Aktivitäten und deren Länge
dfs_small={} #hier kommen alle Fahrzeuge rein
for fzg in range(len(FZG)): #für jedes Fahrzeug
    df = pd.read_excel(path, sheet_name=FZG[fzg]) #Excel aufrufen
    Optionen = ['Stopp', 'Fahren', 'Lenkpause']
    time_format = '%H:%M:%S' 
    tage=df.iloc[2].count() # zählt alle ausgefüllten Tage (zweite zeile)
    tage=tage-1 # da erste Spalte mit Labeln ist
    dfs = {} #hier kommen alle Tage rein
    ende_index = df[df['Fahrzeug'] == 'Ende Uhrzeit'].index[0]
    Schichtende = int(ende_index)
    for tag in range(tage): #für jeden Tage
        column_index = 1 + tag * 2  # This will produce 1, 3, 5, 7, 9, ...
        datum = f'Unnamed: {column_index}'
        
        # Start Uhrzeit
        start='00:00:00'
        start_1=pd.to_datetime(start, format=time_format)
        filtered_df=df[(df['Fahrzeug']=='Start Uhrzeit')& (df[datum].notna())]
        ende=filtered_df.iloc[0, filtered_df.columns.get_loc(datum)]
        ende_1=pd.to_datetime(ende, format=time_format)
        dauer=ende_1-start_1
        data = {
            'Aktivität': [Optionen[0]],
            'Start Time': [start_1],
            'End Time': [ende_1],
            'Duration': [dauer]
        }
        
        df_aktiv = pd.DataFrame(data)
        
        #Anzhal an Stops:
        filtered_df = df[df['Fahrzeug'].str.contains('Ort', case=False, na=False) & df[datum].notna()]
        stopps = filtered_df.shape[0]
        
        i=5 #hier ist der Start zeitpunkt
        # i+4 start erster Lenkpause
        # i+5 Ende erster Lenkpause
        # i+9 start erster Stop
        # i+10 ende erster Stop
        
        while i < ((stopps-1)*9)+5:
            #Subroute einfügen
            if pd.isnull(df.loc[i+4, datum]): # wenn keine Lenkpause: Subroute komplett Einfügen
            #Erste Subroute anders behandeln
                if i==5:
                    start=df.iloc[i, df.columns.get_loc(datum)]
            #Normale Subroute
                else:
                    start=df.iloc[i+1, df.columns.get_loc(datum)]
                start_1=pd.to_datetime(start, format=time_format)
            #Ende des Tages (ganz nach unten springen)
                if pd.isnull(df.loc[i+9, datum]): 
                    ende=df.iloc[Schichtende, df.columns.get_loc(datum)]
                else:
                    ende=df.iloc[i+9, df.columns.get_loc(datum)]
                ende_1=pd.to_datetime(ende, format=time_format)
                dauer= ende_1-start_1
                # Zeile für Matrix erstellen
                new_row_2 = {'Aktivität': Optionen[1], 'Start Time': start_1, 'End Time': ende_1, 'Duration': dauer}
                new_df = pd.DataFrame([new_row_2])
                df_aktiv = pd.concat([df_aktiv, new_df], ignore_index=True)
                
            else: # wenn Lenkpause: 
                start=df.iloc[i+1, df.columns.get_loc(datum)]
                start_r=pd.to_datetime(start, format=time_format)
                #letztes Ende (ganz nach unten springen)
                if pd.isnull(df.loc[i+9, datum]): 
                    ende=df.iloc[Schichtende, df.columns.get_loc(datum)]
                else:
                    ende=df.iloc[i+9, df.columns.get_loc(datum)]
                ende_r=pd.to_datetime(ende, format=time_format)
            
                #Lenkpause berechnen
                start=df.iloc[i+4, df.columns.get_loc(datum)]
                start_lp=pd.to_datetime(start, format=time_format)
                ende=df.iloc[i+5, df.columns.get_loc(datum)]
                ende_lp=pd.to_datetime(ende, format=time_format)
            
            
                #1. Subroute vor Lenkpause 
                dauer=start_lp-start_r
                new_row_2 = {'Aktivität': Optionen[1], 'Start Time': start_r, 'End Time': start_lp, 'Duration': dauer}
                new_df = pd.DataFrame([new_row_2])
                df_aktiv = pd.concat([df_aktiv, new_df], ignore_index=True)
                
                # Lenkpause einfügen
                dauer= ende_lp-start_lp
                new_row_2 = {'Aktivität': Optionen[2], 'Start Time': start_lp, 'End Time': ende_lp, 'Duration': dauer}
                new_df = pd.DataFrame([new_row_2])
                df_aktiv = pd.concat([df_aktiv, new_df], ignore_index=True)
                # Subroute nach Lenkpause
                
                #Subroute geteilt einfügen
                dauer=ende_r-ende_lp
                new_row_2 = {'Aktivität': Optionen[1], 'Start Time': ende_lp, 'End Time': ende_r, 'Duration': dauer}
                new_df = pd.DataFrame([new_row_2])
                df_aktiv = pd.concat([df_aktiv, new_df], ignore_index=True)
                
            # Stop einfügen
            start=df.iloc[i+9, df.columns.get_loc(datum)]
            start_st=pd.to_datetime(start, format=time_format)
            if i==5+(9*(stopps-2)): # letzter stopp
                start=df.iloc[Schichtende, df.columns.get_loc(datum)]
                start_st=pd.to_datetime(start, format=time_format)
                ende='23:59:59'
                ende_st=pd.to_datetime(ende, format=time_format)
                dauer= ende_st-start_st
                new_row_2 = {'Aktivität': Optionen[0], 'Start Time': start_st, 'End Time': ende_st, 'Duration': dauer}
                new_df = pd.DataFrame([new_row_2])
                df_aktiv = pd.concat([df_aktiv, new_df], ignore_index=True)
            else: 
                #jeder anderer Stopp
                start=df.iloc[i+9, df.columns.get_loc(datum)]
                start_st=pd.to_datetime(start, format=time_format)
                ende=df.iloc[i+10, df.columns.get_loc(datum)]
                ende_st=pd.to_datetime(ende, format=time_format)
                dauer= ende_st-start_st
                new_row_2 = {'Aktivität': Optionen[0], 'Start Time': start_st, 'End Time': ende_st, 'Duration': dauer}
                new_df = pd.DataFrame([new_row_2])
                df_aktiv = pd.concat([df_aktiv, new_df], ignore_index=True)
            i+=9
        #Tage in dataframe für Fahrzeug speichern
        dfs[tag]=df_aktiv
    # Alle dataframes eines Fahrzeugs in Liste Speichern
    dfs_small[FZG[fzg]]=dfs
    for key, df in dfs.items():
        df['lang'] = (df['End Time'] - df['Start Time']).dt.total_seconds() / 3600
        df['start_hours'] = df['Start Time'].dt.hour + df['Start Time'].dt.minute / 60 + df['Start Time'].dt.second / 3600

#Safe small Matrix
with open(f'{save_path}dfs_small.pkl', 'wb') as file:
    pickle.dump(dfs_small, file)

In [ ]:
#3. Addressen Matrix 
#3.1. Leere Adressen Matrix und Funktionen
matrix_adressen={}
for fzg in range(len(FZG)):
    name=FZG[fzg]
    matrix_adressen[name]={}
def clean_address(address):
    # Remove multiple spaces
    address = re.sub(r'\s+', ' ', address).strip()
    # Add ", Germany" at the end
    address += ", Germany"
    return address


def geocode_address(address):
    geocode_url = f'https://api.openrouteservice.org/geocode/search?api_key={api_key}&text={address}' #API key wichtig!
    try:
        response = requests.get(geocode_url)
        response.raise_for_status()  # Raise an error for bad status codes
        geocode_data = response.json()
        if 'features' in geocode_data and geocode_data['features']:
            coordinates = geocode_data['features'][0]['geometry']['coordinates']
            return coordinates
        else:
            return 'na'  # Return 'na' if no coordinates found
    except (requests.exceptions.RequestException, KeyError, IndexError) as e:
        # Catch exceptions related to the request or JSON parsing and missing data
        return 'na'

def get_elevations(coordinates):
    # Ensure the coordinates are in the correct format for geojson input
    geojson_coords = {"coordinates": [coordinates], "type": "LineString"}
    elevations = client.elevation_line(
        geometry=geojson_coords,
        format_in='geojson',
        format_out='geojson'
    )
    return elevations


In [ ]:
#3.2 Addressen Matrix befüllen (jedes Fahrzeug einzeln hinzufügen, um zu debuggen & wegen langer Zeit)

FZG_Adresse=FZG[2] # alle Fahrzeuge durchlaufen lassen 0 bis 13


print('Fahrzeug:', FZG_Adresse)
df = pd.read_excel(path, sheet_name=FZG_Adresse)
tage=df.iloc[2].count() # zählt alle ausgefüllten Tage (zweite zeile)
tage=tage-1 # da erste Spalte mit Labeln ist
print('Range Tage:',range(tage))
ende_index = df[df['Fahrzeug'] == 'Ende Uhrzeit'].index[0]
Schichtende = int(ende_index)
number_stop = Schichtende // 9
# Matrix erstellen
matrix_fzg = {}
for tag in range(tage): #für jeden Tage
    column_index = 1 + tag * 2  # This will produce 1, 3, 5, 7, 9, ...
    datum = f'Unnamed: {column_index}'
    print('Spalte:',datum)
    day_key = f'tag {tag}'
    matrix_fzg[day_key] = {}
    #Anzhal an Stops:
    filtered_df = df[df['Fahrzeug'].str.contains('Ort', case=False, na=False) & df[datum].notna()]
    stopps = filtered_df.shape[0]
    print('Stopp Anzahl:',stopps)
    i=5 
    while i < ((stopps)*9)+5:
        #Adresse
        if i==5: #erste Route
            print('erste Route')
            adresse=df.iloc[i-1, df.columns.get_loc(datum)]
            print(adresse)
        else: #alle anderen Routen
            if pd.isnull(df.loc[i, datum]): # letzte Route
                print('letzte Route')
                adresse=df.iloc[ende_index-1, df.columns.get_loc(datum)]
                print(adresse)
            else:
                print('nicht erste route')
                adresse=df.iloc[i-2, df.columns.get_loc(datum)]
                print(adresse)

        adresse = clean_address(adresse)
        print(adresse)
        
        #Coordinaten
        coordinates=geocode_address(adresse)

        
        stop_key = f'stopp{i}'
        matrix_fzg[day_key][stop_key] = {
            'address': adresse,
            'coordinates': coordinates,
        }
        
        i+=9 # Ende der Route
matrix_adressen[FZG_Adresse]=matrix_fzg

In [ ]:
#3.3. Fehler ausbessern und speichern
matrix_adressen['FR-BR 1317']['tag 0']['stopp32']['coordinates']=[7.76421, 48.22380]
matrix_adressen['WT-BR 253']['tag 6']['stopp68']['coordinates']=[7.762176, 48.031589]
matrix_adressen['WT-BR 253']['tag 0']['stopp14']['coordinates']=[7.86854, 48.11022]
matrix_adressen['WT-BR 253']['tag 4']['stopp50']['coordinates']=[7.70276, 47.92384]
matrix_adressen['WT-BR 225']['tag 1']['stopp23']['coordinates']=[8.57074, 49.15896]
matrix_adressen['WT-BR 2551']['tag 5']['stopp14']['coordinates']=[9.07050, 47.69309]
matrix_adressen['WT-BR 228']['tag 3']['stopp59']['coordinates']=[8.03589, 47.80244]
matrix_adressen['WT-BR 228']['tag 2']['stopp32']['coordinates']=[8.52998, 48.0222]
matrix_adressen['WT-BR 270']['tag 3']['stopp68']['coordinates']=[8.56611, 48.00967]
matrix_adressen['WT-BR 270']['tag 3']['stopp59']['coordinates']=[8.16656, 47.9819]
matrix_adressen['WT-BR 270']['tag 2']['stopp23']['coordinates']=[8.45388, 47.81176]
matrix_adressen['WT-BR 270']['tag 2']['stopp14']['coordinates']=[8.45146, 47.81186]
matrix_adressen['WT-BR 229']['tag 8']['stopp32']['coordinates']=[8.03155, 47.80856]
matrix_adressen['WT-BR 229']['tag 8']['stopp41']['coordinates']=[8.03589, 47.80244]
matrix_adressen['WT-BR 608']['tag 7']['stopp14']['coordinates']=[8.63883, 48.35506]
matrix_adressen['WT-BR 213']['tag 3']['stopp59']['coordinates']=[8.16609, 47.84027]
matrix_adressen['WT-BR 229'][f'tag {9}']['stopp23']['coordinates']=[8.17793, 47.63645]
matrix_adressen['WT-BR 213'][f'tag {3}'][f'stopp{41}']['coordinates']=[8.08011,47.49202]
matrix_adressen['WT-BR 213'][f'tag {8}'][f'stopp{14}']['coordinates']=[8.08011,47.49202]
matrix_adressen['WT-BR 231'][f'tag {3}'][f'stopp{32}']['coordinates']=[7.7413,47.54884]
matrix_adressen['WT-BR 231'][f'tag {3}'][f'stopp{41}']['coordinates']=[8.03439,47.56863]
matrix_adressen['WT-BR 231'][f'tag {3}'][f'stopp{23}']['coordinates']=[7.73757,47.55225]
matrix_adressen['WT-BR 231'][f'tag {5}'][f'stopp{32}']['coordinates']=[8.64918,47.92296]
matrix_adressen['WT-BR 270'][f'tag {0}'][f'stopp{32}']['coordinates']=[8.32805,47.81358]
matrix_adressen['WT-BR 270'][f'tag {0}'][f'stopp{50}']['coordinates']=[8.33423,47.81741]
matrix_adressen['WT-BR 270'][f'tag {3}'][f'stopp{68}']['coordinates']=[8.20737,48.051219]
matrix_adressen['WT-BR 2551'][f'tag {0}'][f'stopp{41}']['coordinates']=[7.73757,47.55225]
matrix_adressen['WT-BR 2551'][f'tag {3}'][f'stopp{14}']['coordinates']=[8.142522,48.72869]
matrix_adressen['FR-BR 1317'][f'tag {1}'][f'stopp{23}']['coordinates']=[7.94733,48.21777]
matrix_adressen['FR-BR 1317'][f'tag {3}'][f'stopp{14}']['coordinates']=[7.869919,48.109222]
matrix_adressen['FR-BR 1317'][f'tag {3}'][f'stopp{50}']['coordinates']=[8.010521,48.12133]
matrix_adressen['FR-BR 1317'][f'tag {4}'][f'stopp{59}']['coordinates']=[7.605313,48.09917]
matrix_adressen['WT-BR 213'][f'tag {0}'][f'stopp{14}']['coordinates']=[8.44616,48.08403]
matrix_adressen['WT-BR 213'][f'tag {3}'][f'stopp{41}']['coordinates']=[8.0990403,47.8306168]
matrix_adressen['WT-BR 213'][f'tag {8}'][f'stopp{14}']['coordinates']=[8.0990403,47.8306168]
matrix_adressen['FR-BR 1317'][f'tag {1}'][f'stopp{77}']['coordinates']=[7.695703,47.96743]
with open(f'{save_path}matrix_adressen.pkl', 'wb') as file:
    pickle.dump(matrix_adressen, file)

In [ ]:
#4. Große Matrix erstennen
#4.1. Große Matrix ohne Adressen
for fzg in range(len(FZG)): #für jedes Fahrzeug
    print('Fahrzeug:', FZG[fzg])
    df = pd.read_excel(path, sheet_name=FZG[fzg])
    tage=df.iloc[2].count() # zählt alle ausgefüllten Tage (zweite zeile)
    tage=tage-1 # da erste Spalte mit Labeln ist
    print('Range Tage:',range(tage))
    Optionen = ['Stopp', 'Fahren']
    lenk = ['Ja', 'Nein', '0']
    time_format = '%H:%M:%S' 
    dfs = {} #hier kommen alle Tage rein
    ende_index = df[df['Fahrzeug'] == 'Ende Uhrzeit'].index[0]
    Schichtende = int(ende_index)
    
    for tag in range(tage): #für jeden Tage
        column_index = 1 + tag * 2  # This will produce 1, 3, 5, 7, 9, ...
        datum = f'Unnamed: {column_index}'
        print(datum)
        # Frach start
        filtered_df=df[(df['Fahrzeug']=='Start Ladung (kg)')& (df[datum].notna())]
        fracht_start=filtered_df.iloc[0, filtered_df.columns.get_loc(datum)]
        fracht_aktuell=fracht_start
        # Start Uhrzeit
        start='00:00:00'
        start_1=pd.to_datetime(start, format=time_format)
        filtered_df=df[(df['Fahrzeug']=='Start Uhrzeit')& (df[datum].notna())]
        ende=filtered_df.iloc[0, filtered_df.columns.get_loc(datum)]
        ende_1=pd.to_datetime(ende, format=time_format)
        dauer=ende_1-start_1
        data = {
            'Aktivität': Optionen[0],
            'Start Time': start_1,
            'End Time': ende_1,
            'Duration': dauer,
            'Lenkpause': [lenk[2]],
            'Lenkpause lang': 0,
            'Ort Anfang': 0,
            'Ort Ende': 0,
            'Distanz':0,
            'Geschwindigkeit': 0,
            'Elevation': 0,
            'Fracht':fracht_start,
            'Leergut': 0,
            'Fracht insgs': fracht_start,
            'Abladung': 0
                          
        }
         df_aktiv = pd.DataFrame(data)
        
        #Anzhal an Stops:
        filtered_df = df[df['Fahrzeug'].str.contains('Ort', case=False, na=False) & df[datum].notna()]
        stopps = filtered_df.shape[0]
        
        i=5 #hier ist der Start zeitpunkt
        # i+8 Fracht abladung
        # i+4 start erster Lenkpause
        # i+7 Ort Fahrt Ende
        # i+5 Ende erster Lenkpause
        # i+9 start erster Stop
        # i+10 ende erster Stop
        
        while i < ((stopps-1)*9)+5:
            feste=0
            #Differenzieren nach Subroute mit Lenkpause (für dauer), Subroute ohne Lenkpause, Stopp
            #Erste Route anders behandeln
            if i==5: #erste Route
                print('erste Route')
                zeit_start=df.iloc[i, df.columns.get_loc(datum)]
                fracht_aktuell=fracht_aktuell
                leergut=0
                abladung =0
                fracht_gesamt=fracht_aktuell
            else: #alle anderen Routen
                print('nicht erste route')
                zeit_start=df.iloc[i+1, df.columns.get_loc(datum)]
                abladung= df.iloc[i-1, df.columns.get_loc(datum)]
                feste_abholung= df.iloc[i+2, df.columns.get_loc(datum)]
                if feste_abholung == 'feste': #Feste abholen= +3.500 kg Ladung
                    print('feste')
                    feste_abholung = 3500
                else:
                    print('keine feste')
                    feste = 0
                if abladung < 0: #Neu beladung
                    leergut = 0
                    fracht_aktuell = abladung *(-1)
                    fracht_gesamt=fracht_aktuell 
                    fracht_start=fracht_gesamt
                else:
                    fracht_aktuell = fracht_aktuell - abladung + feste
                    leergut=(fracht_start-fracht_aktuell)*(1/3)
                    fracht_gesamt = fracht_aktuell + leergut #Leergut drauf rechnen
                if fracht_aktuell < 10:
                    fracht_aktuell=0
                else:
                    fracht_aktuell=fracht_aktuell
                    
            #letztes Ende (ganz nach unten springen)
            if pd.isnull(df.loc[i+9, datum]): 
                zeit_ende=df.iloc[Schichtende, df.columns.get_loc(datum)]
            else:
                zeit_ende=df.iloc[i+9, df.columns.get_loc(datum)]

            zeit_start_1=pd.to_datetime(zeit_start, format=time_format)
            zeit_ende_1=pd.to_datetime(zeit_ende, format=time_format)
            zeit_dauer= zeit_ende_1-zeit_start_1

            start_lp=df.iloc[i+4, df.columns.get_loc(datum)]
            start_lp_1=pd.to_datetime(start_lp, format=time_format)
            ende_lp=df.iloc[i+5, df.columns.get_loc(datum)]
            ende_lp_1=pd.to_datetime(ende_lp, format=time_format)
            lenkpause_dauer= ende_lp_1-start_lp_1

            if pd.isnull(df.loc[i+4, datum]):
                #es gibt keine Lenkpause
                lenkpause = 1
                lenkpause_dauer= 0
            else: 
                #Es gibt lenkpause
                lenkpause = 0
                zeit_dauer=zeit_dauer-lenkpause_dauer
                
            new_row  = {
            'Aktivität': Optionen[1],
            'Start Time': zeit_start_1,
            'End Time': zeit_ende_1,
            'Duration': zeit_dauer,
            'Lenkpause': lenk[lenkpause],
            'Lenkpause lang': lenkpause_dauer,
            'Ort Anfang': 0,
            'Ort Ende': 0,
            'Distanz':0,
            'Geschwindigkeit': 0,
            'Elevation': 0,
            'Fracht':fracht_aktuell,
            'Leergut': leergut,
            'Fracht insgs': fracht_gesamt,
            'Abladung': abladung
            }
            new_df = pd.DataFrame([new_row])
            df_aktiv = pd.concat([df_aktiv, new_df], ignore_index=True)
                
            # Stop einfügen
            start=df.iloc[i+9, df.columns.get_loc(datum)]
            start_st=pd.to_datetime(start, format=time_format)
            if i==5+(9*(stopps-2)): # letzter stopp
                start=df.iloc[Schichtende, df.columns.get_loc(datum)]
                start_st=pd.to_datetime(start, format=time_format)
                ende='23:59:59'
                ende_st=pd.to_datetime(ende, format=time_format)
                dauer= ende_st-start_st

            else: #jeder anderer Stopp
                start=df.iloc[i+9, df.columns.get_loc(datum)]
                start_st=pd.to_datetime(start, format=time_format)
                ende=df.iloc[i+10, df.columns.get_loc(datum)]
                ende_st=pd.to_datetime(ende, format=time_format)
                dauer= ende_st-start_st

            new_row_2 = new_row  = {
            'Aktivität': Optionen[0],
            'Start Time': start_st,
            'End Time': ende_st,
            'Duration': dauer,
            'Lenkpause': lenk[2],
            'Lenkpause lang': 0,
            'Ort Anfang': 0,
            'Ort Ende': 0,
            'Distanz':0,
            'Geschwindigkeit':0,
            'Elevation': 0,
            'Fracht':0,
            'Leergut': 0,
            'Abladung': 0
            }
            new_df = pd.DataFrame([new_row_2])
            df_aktiv = pd.concat([df_aktiv, new_df], ignore_index=True)
            i+=9 # Ende der Route
                #den dataframe in einer liste speichern um ihn dann für die Abbildung abzurufen
        dfs[tag]=df_aktiv
    # Alle dataframes eines Fahrzeugs in Liste Speichern
    dfs_big[FZG[fzg]]=dfs



In [ ]:
#4.2. Adressen hinzufügen
def get_elevations(coordinates):
    # Use the "geojson" format for input and output
    elevations = client.elevation_line(
        geometry=coordinates,
        format_in='geojson',
        format_out='geojson'
    )
    return elevations

for fzg in range(len(FZG)): #für jedes Fahrzeug
    print('Fahrzeug:', FZG[fzg])
    df = pd.read_excel(path, sheet_name=FZG[fzg])
    tage=df.iloc[2].count() # zählt alle ausgefüllten Tage (zweite zeile)
    tage=tage-1 # da erste Spalte mit Labeln ist
    print('Range Tage:',range(tage))
    Optionen = ['Stopp', 'Fahren']
    lenk = ['Ja', 'Nein', '0']
    time_format = '%H:%M:%S' 
    dfs = {} #hier kommen alle Tage rein
    ende_index = df[df['Fahrzeug'] == 'Ende Uhrzeit'].index[0]
    Schichtende = int(ende_index)
    
    for tag in range(tage): #für jeden Tage
        column_index = 1 + tag * 2  # This will produce 1, 3, 5, 7, 9, ...
        datum = f'Unnamed: {column_index}'
        print(datum)
        
        #Anzhal an Stops:
        filtered_df = df[df['Fahrzeug'].str.contains('Ort', case=False, na=False) & df[datum].notna()]
        stopps = filtered_df.shape[0]
        a=5
        
        i=1 #hier ist der Start zeitpunkt
        # i+8 Fracht abladung
        # i+4 start erster Lenkpause
        # i+7 Ort Fahrt Ende
        # i+5 Ende erster Lenkpause
        # i+9 start erster Stop
        # i+10 ende erster Stop
        
        while i < (stopps-1)*2:

            print(matrix_adressen[FZG[fzg]][f'tag {tag}'][f'stopp{a}']['address'])
            coords_start=matrix_adressen[FZG[fzg]][f'tag {tag}'][f'stopp{a}']['coordinates']
            print(coords_start)
            coords_ende=matrix_adressen[FZG[fzg]][f'tag {tag}'][f'stopp{a+9}']['coordinates']
            print(coords_ende)

            #Distanz berechnen
            route =client.directions( 
                coordinates=[coords_start, coords_ende],
                profile='driving-car'
                )
            #distanz in km
            if route['routes'][0]['segments'][0]['distance']==0:
                distance = 0
            else:
                distance = route['routes'][0]['summary']['distance'] / 1000
            print('Distanz:',distance)
           #Elevation
            geojson_coords = {
            "type": "LineString",
            "coordinates": [coords_start, coords_ende]
            }
            elevations = get_elevations(geojson_coords)
            diff= elevations['geometry']['coordinates'][1][2]- elevations['geometry']['coordinates'][0][2]
#    dauer_minutes = int(zeit_dauer.total_seconds() / 60)
            print('Elevation:',diff)
            zeit_dauer=dfs_big[FZG[fzg]][tag]['Duration'][i]
            dauer_minutes = int(zeit_dauer.total_seconds() / 60)
            if distance==0:
                geschw =0
            else:  
                if dauer_minutes ==0:
                    geschw=0
                else:
                    geschw = distance/(dauer_minutes/60)
            print('geschwindigkeit:', geschw)
            dfs_big[FZG[fzg]][tag]['Ort Anfang'][i]=matrix_adressen[FZG[fzg]][f'tag {tag}'][f'stopp{a}']['address']
            dfs_big[FZG[fzg]][tag]['Ort Ende'][i]=matrix_adressen[FZG[fzg]][f'tag {tag}'][f'stopp{a+9}']['address']
            dfs_big[FZG[fzg]][tag]['Distanz'][i]=distance
            dfs_big[FZG[fzg]][tag]['Elevation'][i]=diff
            dfs_big[FZG[fzg]][tag]['Geschwindigkeit'][i]=geschw
            print('ende')
            

            a=a+9

#Speichern
with open((f'{save_path}dfs_big.pkl', 'wb') as file:
    pickle.dump(dfs_big, file)

In [ ]:
with open(f'{save_path}dfs_big.pkl', 'rb') as file:   
    dfs_big = pickle.load(file)
with open(f'{save_path}dfs_small.pkl', 'rb') as file:   
    dfs_small = pickle.load(file)
df_kennzahlen = pd.read_excel(f'{save_path}kennzahlen.xlsx')
# dfs_big
with open(f'{save_path}matrix_adressen.pkl', 'rb') as file:   
    matrix_adressen = pickle.load(file)

In [ ]:
#5. Kennzahlen (Durchschnitte, Maxima und Minima)
data = {
            'Name': 'xx',
            'durchschnitt Routenlänge': 0,
            'durchschnit Roudendauer': 0,
            'max Routenlänge': 0,
            'max Routendauer': 0,
            'durchschnitt Stoppanzahl':0,
            'durchschnitt Stopplänge':0,
            'min Stopplänge':0,
            'durchschnitt Lenkpauseanzahl':0,
            'durchschnitt Lenkpauselänge':0,
            'min Lenkpauselänge':0,
            'durchschnitt Subroutenlänge':0,
            'durchschnitt Subroutendauer':0,
            'max Subroutenlänge':0,
            'max Subroutendauer':0,
            'durchschnitt Nacht Länge':0,
            'min Nacht Länge':0,
            'Nr tage':0,
            'Fracht':0,
        }
# Daten für jeden Tag speichern und dann für gesamtes Fahrzeug berechnen        
df_kennzahlen = pd.DataFrame(data, index=['row1'])
df_kennzahlen=df_kennzahlen.drop('row1')
df_kennzahlen
#für jedes Fahzeug Liste mit Tagen erstellen (dauer von fahren, stopps, lenkpausen; Anzahl stopps, lenkpausen; gesamtroutendauer; länge der Nachtpause; max subroutenlänge
for fzg in range(len(FZG)): #für jedes Fahrzeug
    print('Fahrzeug:', FZG[fzg])
    df = pd.read_excel(path, sheet_name=FZG[fzg])
    tage=df.iloc[2].count() # zählt alle ausgefüllten Tage (zweite zeile)
    tage=tage-1 # da erste Spalte mit Labeln ist
    
    f_d_m=[] #fahren dauer durchschnitt
    s_d_m=[] #stopp dauer durchschnitt
    lp_d_m=[] #lenkpause dauer durchschnitt
    s_c=[] #stopp count
    lp_c=[] #lenkpause count
    f_d_max=[] # maximale fahrzeit
    s_d_mini=[] # minimale Stopzeit
    ns_d=[] #dauer Nachtpaus
    route_d=[] #dauer Route
    route_l=[] #länge Route
    f_l_m=[] #fahren länge durchschnitt
    f_l_max=[] # Fahren länge maximum
    n_l=[] #Nachtlänge
    fracht=[] #durchschnittliche Fracht
    
    for tag in range(tage):
        print(tag)
        fahren_durations = dfs_small[FZG[fzg]][tag].loc[dfs_small[FZG[fzg]][tag]['Aktivität'] == 'Fahren', 'Duration']
        print(fahren_durations.mean())
        f_d_m.append(fahren_durations.mean())
        f_d_max.append(fahren_durations.max())
        fahren_lang = dfs_big[FZG[fzg]][tag].loc[dfs_big[FZG[fzg]][tag]['Aktivität'] == 'Fahren', 'Distanz']
        f_l_m.append(fahren_lang.mean())
        f_l_max.append(fahren_lang.max())
        stopp_duration = dfs_small[FZG[fzg]][tag].loc[dfs_small[FZG[fzg]][tag]['Aktivität'] == 'Stopp', 'Duration']
        ns_d.append(stopp_duration[0]+stopp_duration.iat[-1])
        stopp_duration = stopp_duration[1:-1]
        s_d_m.append(stopp_duration.mean())
        s_c.append(stopp_duration.count()) # Anfang und Endstopp raus
        s_d_mini.append(stopp_duration.min())
        fracht_data = dfs_big[FZG[fzg]][tag].loc[dfs_big[FZG[fzg]][tag]['Aktivität'] == 'Fahren', 'Fracht']
        fracht.append(fracht_data.mean())
        if (dfs_small[FZG[fzg]][tag]['Aktivität'] == 'Lenkpause').any():
            lp_duration=dfs_small[FZG[fzg]][tag].loc[dfs_small[FZG[fzg]][tag]['Aktivität'] == 'Lenkpause', 'Duration']
            lp_c.append(lp_duration.count())
        else:
            lp_duration= np.nan
            lp_c.append(0)
        a=np.mean(lp_duration)
        lp_d_m.append(a)
        print(f_d_m, f_d_max)
        nacht=dfs_small[FZG[fzg]][tag]['lang'][len(dfs_small[FZG[fzg]][tag]['lang'])-1]+dfs_small[FZG[fzg]][tag]['lang'][0]
        n_l.append(nacht)
    
        route_d.append(fahren_durations.sum()) #gesamt dauer für route
        route_l.append(fahren_lang.sum())
    # für gesamtes Fahrzeug: Anzahl an Tage, durchschnittdauer fahren, stopps, Lenkpause; max fahrten, min stopps, 
    # min lenkpause; durschnitt häufigkeit von stopps, lenkpausen; min und durchschnitt Nachtpause; durchschnitt und max gesamttour dauer
    f_d_m = pd.Series(f_d_m)
    f_l_m=pd.Series(f_l_m)
    f_l_mean=f_l_m.mean()
    f_d_mean=f_d_m.mean()
    f_d_max=max(f_d_max)
    f_l_max=max(f_l_max)
    tage_c=f_d_m.count()
    s_d_m= pd.Series(s_d_m)
    s_d_mean=s_d_m.mean()
    s_d_mini=pd.Series(s_d_mini)
    s_d_min=s_d_mini.min()
    lp_d_m_list=pd.Series(lp_d_m)
    lp_d_mean=lp_d_m_list.mean()
    lp_d_min=lp_d_m_list.min()
    s_c_mean=np.mean(s_c)
    lp_c_mean=np.mean(lp_c)
    np_mean=np.mean(ns_d)
    ns_d=pd.Series(ns_d)
    np_min=ns_d.min()
    r_d_mean=np.mean(route_d)
    r_l_mean=np.mean(route_l)
    r_d_max=max(route_d)
    r_l_max=max(route_l)
    nacht_lang_mean=np.mean(n_l)
    n_l=pd.Series(n_l)
    nacht_lang_min= n_l.min()
    fracht = pd.Series(fracht)
    fracht_mean=fracht.mean()
    
    new_row  = {
                'Name': new_label[FZG[fzg]],
                'durchschnitt Routenlänge': r_l_mean,
                'durchschnit Roudendauer': r_d_mean,
                'max Routenlänge': r_l_max,
                'max Routendauer': r_d_mean,
                'durchschnitt Stoppanzahl':s_c_mean,
                'durchschnitt Stopplänge':s_d_mean,
                'min Stopplänge':s_d_min,
                'durchschnitt Lenkpauseanzahl':lp_c_mean,
                'durchschnitt Lenkpauselänge':lp_d_mean,
                'min Lenkpauselänge':lp_d_min,
                'durchschnitt Subroutenlänge':f_l_mean,
                'durchschnitt Subroutendauer':f_d_mean,
                'max Subroutenlänge':f_l_max,
                'max Subroutendauer':f_d_max,
                'durchschnitt Nacht Länge':nacht_lang_mean,
                'min Nacht Länge':nacht_lang_min,
                'Nr tage':tage_c,
                'Fracht': fracht_mean
                }
    new_df = pd.DataFrame([new_row])
    df_kennzahlen = pd.concat([df_kennzahlen, new_df], ignore_index=True)
df_kennzahlen
file_name='kennzahlen.xlsx'
full_path = os.path.join(save_path, file_name)
df_kennzahlen.to_excel(full_path, index=False)